In [1]:
import pandas as pd
train= pd.read_csv("train_data.csv",sep=";")
train.head(5)

,origin_latitude,origin_longitude,destination_latitude,destination_longitude,weight,loading_meters,is_adr,shipping_date,cost
0,27.22,105.59,26.91,108.67,0.513,0.195,True,2017-04-24,0.247
1,25.00,109.79,25.34,108.59,0.484,0.195,True,2017-07-27,0.248
2,27.10,105.60,29.35,106.54,0.484,0.195,True,2017-02-03,0.276
3,24.72,106.25,23.43,109.78,0.504,0.195,True,2017-04-03,0.276
4,22.33,108.37,29.38,111.50,0.484,0.195,True,2018-11-08,0.553


In [2]:
train.drop("shipping_date",axis=1,inplace=True)

In [3]:
train.head(3)

,origin_latitude,origin_longitude,destination_latitude,destination_longitude,weight,loading_meters,is_adr,cost
0,27.22,105.59,26.91,108.67,0.513,0.195,True,0.247
1,25.00,109.79,25.34,108.59,0.484,0.195,True,0.248
2,27.10,105.60,29.35,106.54,0.484,0.195,True,0.276


In [4]:
train.is_adr.replace(['True'], [1], inplace=True)


In [6]:
X = train.iloc[:,:-1]

In [8]:
y = train.iloc[:,-1]

In [9]:
X.shape,y.shape

((251155, 7), (251155,))

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
X_train.head(3)

,origin_latitude,origin_longitude,destination_latitude,destination_longitude,weight,loading_meters,is_adr
192604,28.00,112.91,23.58,108.75,0.116,0.195,True
224624,22.45,107.36,29.82,110.52,0.289,0.195,True
46319,23.36,108.30,23.36,109.57,0.484,0.195,True


In [13]:
y_train.head(3)

192604    0.514
224624    0.691
46319     0.248
Name: cost, dtype: float64

In [14]:
X_test.head(3)

,origin_latitude,origin_longitude,destination_latitude,destination_longitude,weight,loading_meters,is_adr
231756,28.39,111.53,22.66,110.35,0.484,0.195,True
100391,26.06,106.17,27.30,110.16,0.210,0.195,True
25222,24.77,110.73,25.22,104.23,0.468,0.195,True


In [15]:
y_test.head(3)

231756    0.311
100391    0.366
25222     0.525
Name: cost, dtype: float64

In [18]:
X_train.shape, y_train.shape,X_test.shape,y_test.shape

((168273, 7), (168273,), (82882, 7), (82882,))

In [19]:
import xgboost
reg = xgboost.XGBRegressor()
reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
y_predict = reg.predict(X_test)

In [21]:
len(y_predict)

82882

In [22]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [23]:
errors_1 = mean_squared_error(y_test, y_predict)
errors_2 = mean_absolute_error(y_test, y_predict)

In [24]:
errors_1

0.003036134177041204

In [25]:
errors_2

0.03467300507090886

In [26]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_predict)
print(r2)

0.8189637478545979


In [35]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score


In [30]:
model_123 = LGBMRegressor()


In [33]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [40]:
n_scores = cross_val_score(model_123, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')

In [41]:
from numpy import mean
from numpy import std
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.038 (0.000)


In [43]:
model_123.fit(X_train, y_train)

LGBMRegressor()

In [44]:
yhat = model_123.predict(X_test)

In [46]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, yhat)
print(r2)

0.7958791994571034
